be really clear about what's going into repaired tables: 
- repaired
- already OK
- unrepairable

Only ~ 215k repaired `env_package`s ?!
- yeah, there just aren't that many annotated in the biosample metadata


- some of this code should probably be moved back into the package
    - splitting and repiping 

- read in previous SSSOMs and don't re-map? or at least double check?

- don't forget to consult "model" field in biosample table too
    - for scoping, in addition to en_package and taxonomy_id?

- what to do about hits against obsolete classes... does OLS provide a mechanism to avoid?
- be careful with GAZ matches
- avoid IDless, OMIT, NCIT, MICRO, BTO

- is it worth trimming and concatenating "x1", "x2", "x3", where x is a mappable term? numbers may be low.
    - example = XXX

- reconcile "x biome" and "x" mappings
- take note of non-local salvage hits
    - like XXX
    - small/large (freshwater) lake?
    - include native labels in the review frame, in addition to local?
    - should use rdftab, not additional OLS calls
    

- switch from hard-coded `env_package_overrides` to something like SSSOM?
- mappings can come from merging strategies, first pass OLS or salvage OLS
- reflect those methods in SSSOM
- add string distance/confidence
    - multiple columns for multiple methods?

review these SSSOM columns

```
    "subject_category": [biosample_col_to_map] * results_rows,
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
```

- are there samples with a host_taxid and a (sample) taxonomy_id from "unclassified"?
- start at uncultured or metagenome taxonomy_id?

- what scoping on or mapping of  "gold" or "path"? WON'T FIND THAT IN NCBI BIOSAMPLE


What to save where? 
- new columns in biosample sqlite? 
- delimited file dumps
- move content from original column and insert the new values into that original column?
- pandas clipboard integration handy for review step


In [1]:
import pandas as pd
import re
import requests as requests
import sqlite3
import string
import urllib
import yaml

import scoped_mapping

from datetime import datetime
from pkg_resources import get_distribution, DistributionNotFound
from strsimpy.cosine import Cosine
from xml.etree import ElementTree
from tdda import rexpy

# User-provided data
See repo README for notes on setting up SQLite databases of OBO ontologies with semantic-sql, relation-graph and rdftab

In [2]:
# from https://www.ncbi.nlm.nih.gov/biosample/docs/packages/?format=xml
# see also https://www.ncbi.nlm.nih.gov/biosample/docs/packages/
biosample_packages_file = "../target/biosample_packages.xml"

# from ftp://ftp.ncbi.nlm.nih.gov//biosample/biosample_set.xml.gz
# via harmonized_table.db.gz
# in https://drive.google.com/drive/u/0/folders/1eL0v0stoduahjDpoDJIk3z2pJBAU4b2Y
biosample_sqlite_file   = "../target/harmonized_table.db"
ncbitaxon_sqlite_file   = "../semantic-sql/db/ncbitaxon.db"


# where do we require a single ontology and where can we use multiple?
target_onto_prefix    = "ENVO"
# 'gaz', 
first_pass_ontologies = ['ncbitaxon']

chars_to_whiteout  = "._-"
my_query_fields    = ""  # OLS weighted default
my_row_req         = 3
my_string_dist_arg = 2
my_max_string_dist = 0.1
salvage_max_dist   = 0.2

## It's nice to see everything accounted for
(In terms of manual mappings for env_package.) But we can prioritize NMDC for now:

- Soil 15,777
- Sediment 7,147
- Plant-associated 3,142

Could some of these "no environmental package" mappings be losing important granularity?

Map `None` and '' to "no environmental package"?

In [3]:
env_package_overrides = {
    "built environment": "built",
    "misc environment": "miscellaneous",
    "missing": "no environmental package",
    "unknown": "no environmental package",
    "default": "no environmental package",
    "unspecified": "no environmental package",
    "not available": "no environmental package",
    "not collected": "no environmental package",
    "miscellaneous natural or artificial environment": "miscellaneous",
    "not applicable": "no environmental package",
    "soil-associated": "soil",
    "soil associated": "soil",
}

## What Biosample field should be mapped to ontology classes?

In [4]:
biosample_col_to_map = "env_medium"

## How should the mapping effort be scoped?
Additional taxonomic filters may be applied below

In [5]:
scoping_col   = "env_package_normalized.EnvPackage"
scoping_value = "soil"

## Settings for manual review of OLS search-based results
Results based on merging Biosample annotations to ontology classes by label or embedded ID are not exported for review at this time

In [6]:
ols_review_file = "../local/ols_review.tsv"
ols_review_seperator = "\t"
strategy_col = "strategy"
include_col  = "include"

first_pass_include_val  = True
first_pass_strategy_val = "env_braod_scale vs envo and gaz @ 0.1"
salvage_include_val     = False
salvage_strategy_val    = "env_braod_scale vs all of ols @ 0.2"

## Settings for SSSOM output

In [7]:
sssom_subject_prefix = "biosample_ebs"
sssom_file           = "biosample_ebs_soil_packages_unclassified_taxa_sssom.tsv"

In [8]:
per_biosample_mapping_results_file = "per_biosample_scoped_mapping_results.tsv"

----

In [9]:
biosample_cnx = sqlite3.connect(biosample_sqlite_file)
ncbitaxon_cnx = sqlite3.connect(ncbitaxon_sqlite_file)
primary_ontology_sqlite_file = "../semantic-sql/db/" + target_onto_prefix.lower() + ".db"
primary_ontology_cnx         = sqlite3.connect(primary_ontology_sqlite_file)
first_pass_ontologies.insert(0, target_onto_prefix.lower())
first_pass_ontologies = ','.join(first_pass_ontologies) 

----

## Print a sample of the data we're working with
Specifically, INSDC/NCBI Biosample metadata

In [10]:
q = """
select
    id,
    env_package,
    package,
    package_name,
    host_taxid,
    taxonomy_id,
    env_broad_scale,
    env_local_scale,
    env_medium
    from biosample b
limit 10
"""
biosample_first_ten = pd.read_sql(q, biosample_cnx)
biosample_first_ten

,id,env_package,package,package_name,host_taxid,taxonomy_id,env_broad_scale,env_local_scale,env_medium
0,BIOSAMPLE:SAMN00000002,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,445970,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
1,BIOSAMPLE:SAMN00000003,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,445972,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
2,BIOSAMPLE:SAMN00000004,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,449673,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
3,BIOSAMPLE:SAMN00000005,None,Generic.1.0,Generic,None,6526,None,None,None
4,BIOSAMPLE:SAMN00000006,None,Generic.1.0,Generic,None,9483,None,None,None
5,BIOSAMPLE:SAMN00000007,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,445974,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
6,BIOSAMPLE:SAMN00000008,missing,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,9606,411461,terrestrial biome [ENVO:00000446],human-associated habitat [ENVO:00009003],biological product [ENVO:02000043]
7,BIOSAMPLE:SAMN00000009,None,Generic.1.0,Generic,None,13616,None,None,None
8,BIOSAMPLE:SAMN00000010,None,Generic.1.0,Generic,None,451639,None,None,None
9,BIOSAMPLE:SAMN00000011,None,Generic.1.0,Generic,None,451638,None,None,None


## Get the canonical checklist and package terms from NCBI

### Could (should?) also examine mixs.yaml (from where?)

Unfortunately it doesn't do a very good job of differentiating checklists (MIMAG, MIMARKS, etc.) from packages (soil, water, etc.)

_What about .ba, .euk, etc?_

In [11]:
package_dictionary = scoped_mapping.get_package_dictionary(biosample_packages_file)
package_dictionary.to_sql(
    "package_dictionary", biosample_cnx, if_exists="replace", index=False
)
package_dictionary

,Name,DisplayName,ShortName,EnvPackage,EnvPackageDisplay,NotAppropriateFor,Description,Example
0,Generic.1.0,Generic,,,,,Generic,
1,SARS-CoV-2.cl.1.0,SARS-CoV-2: clinical or host-associated; versi...,SARS-CoV-2: clinical or host-associated,,,wgs_single;wgs_batch;wgs_diploid,Use for SARS-CoV-2 samples that are relevant t...,
2,Pathogen.cl.1.0,Pathogen: clinical or host-associated; version...,Pathogen: clinical or host-associated,,,,Clinical or host-associated pathogen,SAMN02928182
3,Pathogen.env.1.0,Pathogen: environmental/food/other; version 1.0,Pathogen: environmental/food/other,,,,"Environmental, food or other pathogen",SAMN02730065
4,Microbe.1.0,Microbe; version 1.0,Microbe,,,,Use for bacteria or other unicellular microbes...,SAMN02911891
...,...,...,...,...,...,...,...,...
149,MIUVIG.plant-associated.5.0,"MIUVIG: uncultivated virus genome, plant-assoc...",MIUVIG Uncultivated Virus Genome,plant-associated,plant-associated,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
150,MIUVIG.sediment.5.0,"MIUVIG: uncultivated virus genome, sediment; v...",MIUVIG Uncultivated Virus Genome,sediment,sediment,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
151,MIUVIG.soil.5.0,"MIUVIG: uncultivated virus genome, soil; versi...",MIUVIG Uncultivated Virus Genome,soil,soil,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
152,MIUVIG.wastewater.5.0,"MIUVIG: uncultivated virus genome, wastewater;...",MIUVIG Uncultivated Virus Genome,wastewater,wastewater,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,


## Do the Biosample checklist/package fields match any of the canonical values?

Start by counting the Biosample rows/records. See XXX notes on extracting this *harmonized* database

In [12]:
q = """
select count(*) as biosample_row_count
from biosample b
"""
[biosample_row_count, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=False
)

print(query_duration)
biosample_row_count

0:00:00.062162


,biosample_row_count
0,14300584


## How many of those rows can be inner-joined with the canonical checklists/packages?
Specifically, joining `biosample.package_name` with `package_dictionary.DisplayName`

_Note that indices are built as part of the makefile_


- create index biosample_package_name_idx on biosample(package_name);
- create index package_dictionary_DisplayName_idx on package_dictionary(DisplayName);
- create index biosample_package_idx on biosample(package);
- create index biosample_p_pn_idx on biosample(package, package_name);

In [13]:
q = """
select
    count(*) as canonical_package_name_count
from
    biosample b
inner join package_dictionary pd on
    b.package_name = pd.DisplayName
"""
[canonical_package_name_count, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

print(query_duration)

canonical_package_name_count

2021-06-11 10:59:42.867011
2021-06-11 10:59:43.573479
0:00:00.706468
0:00:00.706468


,canonical_package_name_count
0,14300584


## Combinations of `package` and `package_name` values in the Biosample dataset

In [14]:
q = """
select
    package,
    package_name,
    count(*) as count
from
    biosample b
group by
    package ,
    package_name
order by
    package ,
    package_name
"""
[package_name_combos, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

print(query_duration)

package_name_combos

2021-06-11 10:59:43.582597
2021-06-11 10:59:44.958328
0:00:01.375731
0:00:01.375731


,package,package_name,count
0,Beta-lactamase.1.0,Beta-lactamase; version 1.0,556
1,Generic.1.0,Generic,10186430
2,Human.1.0,Human; version 1.0,368893
3,Invertebrate.1.0,Invertebrate; version 1.0,131085
4,MIGS.ba.5.0,MIGS: cultured bacteria/archaea; version 5.0,28686
...,...,...,...
124,Model.organism.animal.1.0,Model organism or animal; version 1.0,382980
125,Pathogen.cl.1.0,Pathogen: clinical or host-associated; version...,511040
126,Pathogen.env.1.0,Pathogen: environmental/food/other; version 1.0,246497
127,Plant.1.0,Plant; version 1.0,385939


## What about the Biosample `env_package` values?
Are they also a small, highly regular set, like the `package` and `package_name` combinations?

In [15]:
q = """
select
    env_package,
    count(*) as count
from
    biosample b
group by
    env_package
order by
    count(*) desc
"""
[env_package_count, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

env_package_count

0:00:00.819255


,env_package,count
0,None,14083847
1,host-associated,49254
2,human-gut,47921
3,water,16367
4,human-skin,13706
...,...,...
87,env_package,1
88,gut microbiome,1
89,marine sediment (ENVO:00002113),1
90,saliva,1


## `env_package` is a mixture of `ENVO` term ids and strings
Those strings may or may not be the term's labels. There are many redundancies due to small spelling and punctuation variations.

Start by tidying `env_package` etc. values recognized by INSDC

In [16]:
package_dictionary = scoped_mapping.make_tidy_col(
    package_dictionary, "EnvPackage", "eptidy"
)
package_dictionary = scoped_mapping.make_tidy_col(
    package_dictionary, "EnvPackageDisplay", "epdtidy"
)

# update in sqlite
package_dictionary.to_sql(
    "package_dictionary", biosample_cnx, if_exists="replace", index=False
)

## Determine ID patterns for common ontologies, like `ENVO`

In [17]:
q = """
select
    distinct stanza
    from statements s
where
    predicate = 'rdf:type'
    and "object" = 'owl:Class'
    and stanza = subject"""
# include non-envo IDs that come from envo?
[ids_from_selected_ontolgy, query_duration] = scoped_mapping.timed_query(
    q, primary_ontology_cnx
)
print(query_duration)

0:00:00.020218


In [18]:
ids_from_selected_ontolgy = scoped_mapping.add_prefix_col(
    ids_from_selected_ontolgy, "stanza", "prefix"
)

In [19]:
id_patterns = scoped_mapping.get_multi_term_patterns(
    ids_from_selected_ontolgy, "stanza", "prefix"
)

id_patterns

{'BFO': 'BFO:\\d{7}',
 'CARO': 'CARO:\\d{7}',
 'CHEBI': 'CHEBI:\\d{4,6}',
 'ENVO': 'ENVO:\\d{7,8}',
 'FAO': 'FAO:0000001',
 'FOODON': 'FOODON:\\d{8}',
 'GO': 'GO:\\d{7}',
 'IAO': 'IAO:\\d{7}',
 'NCBITaxon': 'NCBITaxon:\\d+',
 'OBI': 'OBI:\\d{7}',
 'PATO': 'PATO:\\d{7}',
 'PCO': 'PCO:\\d{7}',
 'PO': 'PO:\\d{7}',
 'RO': 'RO:0002577',
 'UBERON': 'UBERON:\\d{7}'}

## Apply some normalization rules to the `env_package` values

In [20]:
env_package_normalized = scoped_mapping.env_package_nomralizastion(
    env_package_count, "env_package", id_patterns[target_onto_prefix]
)

# getting rid of redundant? 'string' column
env_package_normalized = env_package_normalized[
    [
        "env_package",
        "count",
        "lhs",
        "rhs",
        "extract",
        "remaining_string",
        "remaining_tidied",
    ]
]

env_package_normalized

,env_package,count,lhs,rhs,extract,remaining_string,remaining_tidied
0,None,14083847,,None,,,
1,host-associated,49254,,host-associated,,host-associated,host associated
2,human-gut,47921,,human-gut,,human-gut,human gut
3,water,16367,,water,,water,water
4,human-skin,13706,,human-skin,,human-skin,human skin
...,...,...,...,...,...,...,...
87,env_package,1,,env_package,,env_package,env package
88,gut microbiome,1,,gut microbiome,,gut microbiome,gut microbiome
89,marine sediment (ENVO:00002113),1,,marine sediment (ENVO:00002113),ENVO:00002113,marine sediment (),marine sediment
90,saliva,1,,saliva,,saliva,saliva


pick "EnvPackage"/"eptidy" from package dictionary as canonical, not "EnvPackageDisplay"/"epdtidy"


But still want to support making XXX values from YYY canonical according to "EnvPackageDisplay"/"epdtidy", so make a mapping/override table

In [21]:
epd_to_ep = package_dictionary[["eptidy", "epdtidy"]]
# drop duplicates
epd_to_ep = epd_to_ep.drop_duplicates()

# drop blank eptidy rows
ep_blank_flag = epd_to_ep["eptidy"].eq("")
epd_to_ep = epd_to_ep.loc[~ep_blank_flag]

# drop rows where eptidy and epdtidy are the same
identical_flag = epd_to_ep["eptidy"] == epd_to_ep["epdtidy"]
epd_to_ep = epd_to_ep.loc[~identical_flag]

epd_to_ep

,eptidy,epdtidy
21,microbial,microbial mat biofilm
22,miscellaneous,miscellaneous or artificial
26,wastewater,wastewater sludge


In [22]:
# and add to manualy asserted overrides above
overrides_supplement = dict(zip(epd_to_ep["epdtidy"], epd_to_ep["eptidy"]))

overrides_supplement

{'microbial mat biofilm': 'microbial',
 'miscellaneous or artificial': 'miscellaneous',
 'wastewater sludge': 'wastewater'}

In [23]:
# not getting soil-associated
env_package_overrides.update(overrides_supplement)

env_package_overrides

{'built environment': 'built',
 'misc environment': 'miscellaneous',
 'missing': 'no environmental package',
 'unknown': 'no environmental package',
 'default': 'no environmental package',
 'unspecified': 'no environmental package',
 'not available': 'no environmental package',
 'not collected': 'no environmental package',
 'miscellaneous natural or artificial environment': 'miscellaneous',
 'not applicable': 'no environmental package',
 'soil-associated': 'soil',
 'soil associated': 'soil',
 'microbial mat biofilm': 'microbial',
 'miscellaneous or artificial': 'miscellaneous',
 'wastewater sludge': 'wastewater'}

In [24]:
env_package_normalized = scoped_mapping.add_overrides(
    env_package_normalized, "remaining_tidied", "rt_override", env_package_overrides
)

In [25]:
denorm_frame = package_dictionary[["EnvPackage", "eptidy"]]
denorm_frame = denorm_frame.drop_duplicates()
denorm_frame

,EnvPackage,eptidy
0,,
12,No environmental package,no environmental package
13,air,air
14,built,built
15,host-associated,host associated
16,human-associated,human associated
17,human-gut,human gut
18,human-oral,human oral
19,human-skin,human skin
20,human-vaginal,human vaginal


In [26]:
env_package_normalized = env_package_normalized.merge(
    denorm_frame, how="left", left_on="rt_override", right_on="eptidy"
)

env_package_normalized = env_package_normalized[
    [
        "env_package",
        "count",
        "lhs",
        "rhs",
        "extract",
        "remaining_string",
        "remaining_tidied",
        "rt_override",
        "EnvPackage",
    ]
]

non_canonical_flag = env_package_normalized["EnvPackage"].isna()
env_package_normalized["is_canonical"] = True
env_package_normalized.loc[non_canonical_flag, "is_canonical"] = False


- env_package = env_package annotation from NCBI Biosample file XXX
- count = number of biosamples using that env_package annotation
- lhs = checklist info
- rhs = potential package info
- extract = potential OBO ID from rhs column (currently harcoded and only looking for ENVO IDs)
- remaining_string = rhs/string, with potential OBO IDs removed
- remaining_tidied = remaining_string with case, whitespace and punctuation normailzastion
- rt_override = some remaining_tidied values can be replaced according to env_package_overrides
- EnvPackage = corresponding de-normalized value from package_dictionary
- is_canonical = false when EnvPackage is NaN

In [27]:
env_package_normalized.to_sql(
    "env_package_normalized", biosample_cnx, if_exists="replace", index=False
)

env_package_normalized.to_csv(
    "biosample_env_package_normalizastion.tsv", sep=ols_review_seperator, index=False
)

In [28]:
env_package_normalized

,env_package,count,lhs,rhs,extract,remaining_string,remaining_tidied,rt_override,EnvPackage,is_canonical
0,None,14083847,,None,,,,,,True
1,host-associated,49254,,host-associated,,host-associated,host associated,host associated,host-associated,True
2,human-gut,47921,,human-gut,,human-gut,human gut,human gut,human-gut,True
3,water,16367,,water,,water,water,water,water,True
4,human-skin,13706,,human-skin,,human-skin,human skin,human skin,human-skin,True
...,...,...,...,...,...,...,...,...,...,...
87,env_package,1,,env_package,,env_package,env package,env package,NaN,False
88,gut microbiome,1,,gut microbiome,,gut microbiome,gut microbiome,gut microbiome,NaN,False
89,marine sediment (ENVO:00002113),1,,marine sediment (ENVO:00002113),ENVO:00002113,marine sediment (),marine sediment,marine sediment,NaN,False
90,saliva,1,,saliva,,saliva,saliva,saliva,NaN,False


## What do the successful normalizations look like?

In [29]:
q = """
select
    env_package,
    count,
    lhs,
    extract,
    EnvPackage
from
    env_package_normalized
where
    is_canonical = 1
"""
[successful_normalizastions, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx
)

print(query_duration)

successful_normalizastions

0:00:00.001594


,env_package,count,lhs,extract,EnvPackage
0,None,14083847,,,
1,host-associated,49254,,,host-associated
2,human-gut,47921,,,human-gut
3,water,16367,,,water
4,human-skin,13706,,,human-skin
...,...,...,...,...,...
57,not collected,3,,,No environmental package
58,host associated,2,,,host-associated
59,ENVO:00000016,1,,ENVO:00000016,
60,Sediment,1,,,sediment


# Are there any normalization failures?

In [30]:
q = """
select
    env_package,
    count,
    lhs,
    extract,
    EnvPackage
from
    env_package_normalized
where
    is_canonical = 0
"""
[normalizastion_failures, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

normalizastion_failures

0:00:00.001350


,env_package,count,lhs,extract,EnvPackage
0,mimarks,479,,,None
1,mouse-gut,406,,,None
2,gut,172,,,None
3,biofilm,114,,,None
4,human-not providedsopharyngeal,107,,,None
5,mice gut,87,,,None
6,CV,60,,,None
7,"home, outdoor environment",44,,,None
8,fermentation-associated,42,,,None
9,sterile water,35,,,None


In [31]:
cursor = biosample_cnx.cursor()
statement = """
DROP TABLE if exists repaired_env_package ;

CREATE TABLE repaired_env_package AS
select
	b.id,
	epn.env_package as env_package_orig,
	epn.EnvPackage as env_package_rep
from
	env_package_normalized epn
join biosample b on
	b.env_package = epn.env_package
where
	is_canonical = 1
	and EnvPackage != '';
	
select
	count(*)
from
	repaired_env_package rep
"""

cursor.executescript(statement)
biosample_cnx.commit()


# Utilizing taxonomy for broad subsetting


**This uses an SQLite database in which the transitive closure over subClassOf has already been materialized. See the README and Makefile.**

Specifically, flag the Biosamples whose `taxon_id` indicates they are an unclassified entity. Ignoring the others will throw out samples of multicellular organisms, like fruit flies.



## Get a listing of all taxa that are transitive subclasses of `NCBITaxon:2787823`

I.e. 'unclassified entities'

In [32]:
q = """
select
    distinct s.subject
from
    entailed_edge ee
join statements s on
    ee.subject = s.subject
where
    ee.predicate = 'rdfs:subClassOf'
    and ee.object = 'NCBITaxon:2787823'
    and s.predicate = 'rdfs:label'
"""
[unclassified_taxa, query_duration] = scoped_mapping.timed_query(q, ncbitaxon_cnx)
unclassified_taxa["unclassified"] = True

print(query_duration)

unclassified_taxa

0:00:06.176164


,subject,unclassified
0,NCBITaxon:1006967,True
1,NCBITaxon:1041057,True
2,NCBITaxon:1046002,True
3,NCBITaxon:1046003,True
4,NCBITaxon:1046004,True
...,...,...
989,NCBITaxon:939928,True
990,NCBITaxon:941420,True
991,NCBITaxon:941421,True
992,NCBITaxon:941422,True


## Get taxon counts from the Biosample metadata

In [33]:
q = """
select
    taxonomy_id biosample_taxid,
    count(*) as count
from
    biosample b
group by
    taxonomy_id
order by
    count(*) desc
"""
[biosample_tax_id_counts, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)
biosample_tax_id_counts["curie"] = "NCBITaxon:" + biosample_tax_id_counts[
    "biosample_taxid"
].astype(str)

print(query_duration)

0:00:01.172136


## Merge the two taxonomy dataframes

I.e. flag the the Biosample records whose `taxonomy_id` field belongs to a subclass of 'unclassified entries'.

In [34]:
biosample_tax_id_counts = biosample_tax_id_counts.merge(
    unclassified_taxa, left_on="curie", right_on="subject", how="left"
)
biosample_tax_id_counts.unclassified.fillna(False, inplace=True)

biosample_tax_id_counts

,biosample_taxid,count,curie,subject,unclassified
0,9606,6819707,NCBITaxon:9606,NaN,False
1,10090,964219,NCBITaxon:10090,NaN,False
2,408170,290862,NCBITaxon:408170,NCBITaxon:408170,True
3,410658,280666,NCBITaxon:410658,NCBITaxon:410658,True
4,646099,208741,NCBITaxon:646099,NCBITaxon:646099,True
...,...,...,...,...,...
163372,999891,1,NCBITaxon:999891,NaN,False
163373,999892,1,NCBITaxon:999892,NaN,False
163374,999898,1,NCBITaxon:999898,NaN,False
163375,999931,1,NCBITaxon:999931,NaN,False


## Add labels to all taxa

In [35]:
q = """
select
    subject ,
    value
from statements
where
    predicate = 'rdfs:label' and subject = stanza
"""
[all_tax_labels, query_duration] = scoped_mapping.timed_query(q, ncbitaxon_cnx)

biosample_tax_id_counts = biosample_tax_id_counts.merge(
    all_tax_labels, left_on="curie", right_on="subject", how="left"
)

biosample_tax_id_counts = biosample_tax_id_counts[
    ["curie", "biosample_taxid", "count", "unclassified", "value"]
]
biosample_tax_id_counts.columns = [
    "curie",
    "biosample_taxid",
    "count",
    "unclassified",
    "label",
]

print(query_duration)
biosample_tax_id_counts.to_sql(
    "biobiosample_tax_id_counts", biosample_cnx, if_exists="replace", index=False
)

biosample_tax_id_counts

0:00:03.276666


,curie,biosample_taxid,count,unclassified,label
0,NCBITaxon:9606,9606,6819707,False,Homo sapiens
1,NCBITaxon:10090,10090,964219,False,Mus musculus
2,NCBITaxon:408170,408170,290862,True,human gut metagenome
3,NCBITaxon:410658,410658,280666,True,soil metagenome
4,NCBITaxon:646099,646099,208741,True,human metagenome
...,...,...,...,...,...
163372,NCBITaxon:999891,999891,1,False,Bacillus amyloliquefaciens TA208
163373,NCBITaxon:999892,999892,1,False,[Propionibacterium] humerusii P08
163374,NCBITaxon:999898,999898,1,False,Peptococcaceae bacterium CEB3
163375,NCBITaxon:999931,999931,1,False,Barrientosiimonas humi


**Almost all of the taxa that are common in the biosample collection are either unclassified/metagenomes or easily recognized cellular organisms. Cellular organism samples are de-prioritized in this exercise**

Exceptions include:
- 32630 = synthetic construct (other entries; other sequences; artificial sequences)
    - 'other entries' would add 16k rows on top of the 1k 'unclassified entities'
    - metagenomes account for 331 of the 'unclassified entities'
    - there are also a small number of uncultured/unclassified microorganisms in the biosample dataset
- 77133 = uncultured bacterium (cellular organisms; Bacteria; environmental samples)
    - 'cellular organisms' would add 2M rows on top of the 1k 'unclassified entities'
    - 'cellular organisms; Bacteria; environmental samples' adds 26k
    
----

## Get a table of MIxS annotations to be mapped to ontology classes.

Explicitly scope based on normalized package data. These values were set at the top of this notebook.


In [36]:
print(biosample_col_to_map)
print(scoping_col)
print(scoping_value)

env_medium
env_package_normalized.EnvPackage
soil


**In this case, the scoping includes an inner join requirement for 'unclassified entities'**

In [37]:
q = (
    "select "
    + biosample_col_to_map
    + """, count(*) as count
from
    biosample b
join env_package_normalized on
    b.env_package = env_package_normalized.env_package
inner join biobiosample_tax_id_counts stic on
    b.taxonomy_id = stic.biosample_taxid
where """
    + scoping_col
    + " = '"
    + scoping_value
    + "' group by "
    + biosample_col_to_map
    + """
order by
    count(*) desc"""
)

[mapping_candidates, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

mapping_candidates

,env_medium,count
0,soil,12400
1,Soil,439
2,rhizosphere,238
3,peat soil,224
4,ENVO:soil,214
...,...,...
101,4,1
102,24,1
103,22,1
104,15,1


----

## The Biosample format allows for pipe-delimited environmental package lists. 

Separate those out into their components.

----


In [38]:
multi_frames = []
for row in mapping_candidates.itertuples(index=True, name="PandaZ"):
    split_check = row[1]
    if split_check is None:
        split_check = ""
    splitted = pd.Series(split_check.split("|"))
    splitted_count = len(splitted)
    repeated = [split_check] * splitted_count
    repeated = pd.Series(repeated)
    as_frame = pd.DataFrame(dict(repeated=repeated, splitted=splitted)).reset_index()
    seq_list = list(range(1, splitted_count + 1))
    as_frame["part_count"] = splitted_count
    as_frame["seq"] = seq_list
    multi_frames.append(as_frame)
concat_frame = pd.concat(multi_frames)
concat_frame = concat_frame[["repeated", "splitted", "part_count", "seq"]]
concat_frame

,repeated,splitted,part_count,seq
0,soil,soil,1,1
0,Soil,Soil,1,1
0,rhizosphere,rhizosphere,1,1
0,peat soil,peat soil,1,1
0,ENVO:soil,ENVO:soil,1,1
...,...,...,...,...
0,4,4,1,1
0,24,24,1,1
0,22,22,1,1
0,15,15,1,1


In [39]:
mapping_candidates = mapping_candidates.merge(
    concat_frame, left_on=biosample_col_to_map, right_on="repeated", how="left"
)

mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq
0,soil,12400,soil,soil,1.0,1.0
1,Soil,439,Soil,Soil,1.0,1.0
2,rhizosphere,238,rhizosphere,rhizosphere,1.0,1.0
3,peat soil,224,peat soil,peat soil,1.0,1.0
4,ENVO:soil,214,ENVO:soil,ENVO:soil,1.0,1.0
...,...,...,...,...,...,...
101,4,1,4,4,1.0,1.0
102,24,1,24,24,1.0,1.0
103,22,1,22,22,1.0,1.0
104,15,1,15,15,1.0,1.0


## Normalize a few different ways `ENVO` IDs have been entered
In the Biosample metadata

In [40]:
# this helps by standardizing what could be an ID
# but if the RHS is text, then envo should just be removed
mapping_candidates["term_tidy"] = mapping_candidates.splitted.str.replace(
    target_onto_prefix.lower() + "[:_ ]",
    target_onto_prefix + ":",
    regex=True,
    case=False,
)

mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy
0,soil,12400,soil,soil,1.0,1.0,soil
1,Soil,439,Soil,Soil,1.0,1.0,Soil
2,rhizosphere,238,rhizosphere,rhizosphere,1.0,1.0,rhizosphere
3,peat soil,224,peat soil,peat soil,1.0,1.0,peat soil
4,ENVO:soil,214,ENVO:soil,ENVO:soil,1.0,1.0,ENVO:soil
...,...,...,...,...,...,...,...
101,4,1,4,4,1.0,1.0,4
102,24,1,24,24,1.0,1.0,24
103,22,1,22,22,1.0,1.0,22
104,15,1,15,15,1.0,1.0,15


# Now try to extract ontology terms that are already present

In [41]:
candidate_series_decomposition = scoped_mapping.decompose_series(
    mapping_candidates["term_tidy"], id_patterns[target_onto_prefix]
)

mapping_candidates = pd.concat(
    [mapping_candidates, candidate_series_decomposition], axis=1
)

mapping_candidates[
    "remaining_tidied"
] = mapping_candidates.remaining_tidied.str.replace(
    target_onto_prefix.lower() + "[:_ ]", "", regex=True, case=False
)

mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied
0,soil,12400,soil,soil,1.0,1.0,soil,soil,,soil,soil
1,Soil,439,Soil,Soil,1.0,1.0,Soil,Soil,,Soil,soil
2,rhizosphere,238,rhizosphere,rhizosphere,1.0,1.0,rhizosphere,rhizosphere,,rhizosphere,rhizosphere
3,peat soil,224,peat soil,peat soil,1.0,1.0,peat soil,peat soil,,peat soil,peat soil
4,ENVO:soil,214,ENVO:soil,ENVO:soil,1.0,1.0,ENVO:soil,ENVO:soil,,ENVO:soil,soil
...,...,...,...,...,...,...,...,...,...,...,...
101,4,1,4,4,1.0,1.0,4,4,,4,4
102,24,1,24,24,1.0,1.0,24,24,,24,24
103,22,1,22,22,1.0,1.0,22,22,,22,22
104,15,1,15,15,1.0,1.0,15,15,,15,15


## Join the extracted IDs with their labels

_Start by connecting to the rdftab database from which the terms and label-like annotations will be obtained?_

## extracting the labels

In [42]:
q = """
select
    subject ,
    value
from
    statements s
where
    predicate = 'rdfs:label'
"""

[onto_labels, query_duration] = scoped_mapping.timed_query(q, primary_ontology_cnx)

onto_labels

,subject,value
0,IAO:0000111,editor preferred term~editor preferred label
1,IAO:0000112,example of usage
2,IAO:0000114,has curation status
3,IAO:0000115,definition
4,IAO:0000116,editor note
...,...,...
6774,ENVO:01001862,Solar radiation
6775,<https://www.wikidata.org/wiki/Q2>,Earth
6776,<https://www.wikidata.org/wiki/Q2306597>,Suni
6777,<https://www.wikidata.org/wiki/Q525>,Sol


## and merging 

In [43]:
mapping_candidates = mapping_candidates.merge(
    onto_labels, left_on="extract", right_on="subject", how="left"
)
mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,subject,value
0,soil,12400,soil,soil,1.0,1.0,soil,soil,,soil,soil,NaN,NaN
1,Soil,439,Soil,Soil,1.0,1.0,Soil,Soil,,Soil,soil,NaN,NaN
2,rhizosphere,238,rhizosphere,rhizosphere,1.0,1.0,rhizosphere,rhizosphere,,rhizosphere,rhizosphere,NaN,NaN
3,peat soil,224,peat soil,peat soil,1.0,1.0,peat soil,peat soil,,peat soil,peat soil,NaN,NaN
4,ENVO:soil,214,ENVO:soil,ENVO:soil,1.0,1.0,ENVO:soil,ENVO:soil,,ENVO:soil,soil,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,4,1,4,4,1.0,1.0,4,4,,4,4,NaN,NaN
102,24,1,24,24,1.0,1.0,24,24,,24,24,NaN,NaN
103,22,1,22,22,1.0,1.0,22,22,,22,22,NaN,NaN
104,15,1,15,15,1.0,1.0,15,15,,15,15,NaN,NaN


## Use cosine string distance to see if the labels match closely enough

I.e. the labels claimed by the Biosample data set and the labels asserted in the ontology. if they're close enough, consider the assigned ID legit


_How close is close enough?_

In [44]:
my_cosine_obj = Cosine(my_string_dist_arg)
mapping_candidates["value"] = mapping_candidates["value"].fillna("")
mapping_candidates["remaining_tidied"] = mapping_candidates["remaining_tidied"].fillna(
    ""
)
mapping_candidates["cosine"] = mapping_candidates.apply(
    lambda my_row: my_cosine_obj.distance(
        my_row["remaining_tidied"].lower(), my_row["value"].lower()
    ),
    axis=1,
)
mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,subject,value,cosine
0,soil,12400,soil,soil,1.0,1.0,soil,soil,,soil,soil,NaN,,1.0
1,Soil,439,Soil,Soil,1.0,1.0,Soil,Soil,,Soil,soil,NaN,,1.0
2,rhizosphere,238,rhizosphere,rhizosphere,1.0,1.0,rhizosphere,rhizosphere,,rhizosphere,rhizosphere,NaN,,1.0
3,peat soil,224,peat soil,peat soil,1.0,1.0,peat soil,peat soil,,peat soil,peat soil,NaN,,1.0
4,ENVO:soil,214,ENVO:soil,ENVO:soil,1.0,1.0,ENVO:soil,ENVO:soil,,ENVO:soil,soil,NaN,,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,4,1,4,4,1.0,1.0,4,4,,4,4,NaN,,1.0
102,24,1,24,24,1.0,1.0,24,24,,24,24,NaN,,1.0
103,22,1,22,22,1.0,1.0,22,22,,22,22,NaN,,1.0
104,15,1,15,15,1.0,1.0,15,15,,15,15,NaN,,1.0


In [45]:
mapping_candidates.to_clipboard(index=False)

**Previously, we did a reality check on the claimed IDs and labels. If a label is claimed without any ID, that could still be a path to an ontology term.**

We'll be doing some merging, so make sure column names are meaningful and unique


In [46]:
mapping_candidates.columns = [
    biosample_col_to_map,
    "count",
    "repeated",
    "splitted",
    "part_count",
    "seq",
    "term_tidy",
    "string",
    "extract",
    "remaining_string",
    "remaining_tidied",
    "term_id",
    "lab_from_id",
    "lfi_cosine",
]
mapping_candidates = mapping_candidates.merge(
    onto_labels, left_on="remaining_tidied", right_on="value", how="left"
)

mapping_candidates.columns = [
    biosample_col_to_map,
    "count",
    "repeated",
    "splitted",
    "part_count",
    "seq",
    "term_tidy",
    "string",
    "extract",
    "remaining_string",
    "remaining_tidied",
    "term_id",
    "lab_from_id",
    "lfi_cosine",
    "term_id_from_lab",
    "value",
]


mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,term_id,lab_from_id,lfi_cosine,term_id_from_lab,value
0,soil,12400,soil,soil,1.0,1.0,soil,soil,,soil,soil,NaN,,1.0,ENVO:00001998,soil
1,Soil,439,Soil,Soil,1.0,1.0,Soil,Soil,,Soil,soil,NaN,,1.0,ENVO:00001998,soil
2,rhizosphere,238,rhizosphere,rhizosphere,1.0,1.0,rhizosphere,rhizosphere,,rhizosphere,rhizosphere,NaN,,1.0,ENVO:00005801,rhizosphere
3,peat soil,224,peat soil,peat soil,1.0,1.0,peat soil,peat soil,,peat soil,peat soil,NaN,,1.0,ENVO:00005774,peat soil
4,ENVO:soil,214,ENVO:soil,ENVO:soil,1.0,1.0,ENVO:soil,ENVO:soil,,ENVO:soil,soil,NaN,,1.0,ENVO:00001998,soil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,4,1,4,4,1.0,1.0,4,4,,4,4,NaN,,1.0,NaN,NaN
102,24,1,24,24,1.0,1.0,24,24,,24,24,NaN,,1.0,NaN,NaN
103,22,1,22,22,1.0,1.0,22,22,,22,22,NaN,,1.0,NaN,NaN
104,15,1,15,15,1.0,1.0,15,15,,15,15,NaN,,1.0,NaN,NaN


## Find consensus term IDs and labels


In [47]:
mapping_candidates["consensus_id"] = mapping_candidates["term_id_from_lab"]
mapping_candidates["consensus_lab"] = mapping_candidates["value"]

flag = mapping_candidates["consensus_id"].isnull() & (
    (
        ~mapping_candidates["term_id"].isnull()
        & mapping_candidates.lfi_cosine.le(my_max_string_dist)
    )
    | (
        ~mapping_candidates["term_id"].isnull()
        & mapping_candidates["remaining_tidied"].eq("")
    )
)

replacements = mapping_candidates.loc[flag, "term_id"]
mapping_candidates.loc[flag, "consensus_id"] = replacements

replacements = mapping_candidates.loc[flag, "lab_from_id"]
mapping_candidates.loc[flag, "consensus_lab"] = replacements

flag = mapping_candidates.consensus_id.isna()
antiflag = ~flag
mapping_candidates["id_or_lab_ok"] = antiflag

mapping_candidates["assembled_consensus"] = (
    mapping_candidates["consensus_lab"]
    + " ["
    + mapping_candidates["consensus_id"]
    + "]"
)

In [48]:
mapping_candidates

,env_medium,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,term_id,lab_from_id,lfi_cosine,term_id_from_lab,value,consensus_id,consensus_lab,id_or_lab_ok,assembled_consensus
0,soil,12400,soil,soil,1.0,1.0,soil,soil,,soil,soil,NaN,,1.0,ENVO:00001998,soil,ENVO:00001998,soil,True,soil [ENVO:00001998]
1,Soil,439,Soil,Soil,1.0,1.0,Soil,Soil,,Soil,soil,NaN,,1.0,ENVO:00001998,soil,ENVO:00001998,soil,True,soil [ENVO:00001998]
2,rhizosphere,238,rhizosphere,rhizosphere,1.0,1.0,rhizosphere,rhizosphere,,rhizosphere,rhizosphere,NaN,,1.0,ENVO:00005801,rhizosphere,ENVO:00005801,rhizosphere,True,rhizosphere [ENVO:00005801]
3,peat soil,224,peat soil,peat soil,1.0,1.0,peat soil,peat soil,,peat soil,peat soil,NaN,,1.0,ENVO:00005774,peat soil,ENVO:00005774,peat soil,True,peat soil [ENVO:00005774]
4,ENVO:soil,214,ENVO:soil,ENVO:soil,1.0,1.0,ENVO:soil,ENVO:soil,,ENVO:soil,soil,NaN,,1.0,ENVO:00001998,soil,ENVO:00001998,soil,True,soil [ENVO:00001998]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,4,1,4,4,1.0,1.0,4,4,,4,4,NaN,,1.0,NaN,NaN,NaN,NaN,False,NaN
102,24,1,24,24,1.0,1.0,24,24,,24,24,NaN,,1.0,NaN,NaN,NaN,NaN,False,NaN
103,22,1,22,22,1.0,1.0,22,22,,22,22,NaN,,1.0,NaN,NaN,NaN,NaN,False,NaN
104,15,1,15,15,1.0,1.0,15,15,,15,15,NaN,,1.0,NaN,NaN,NaN,NaN,False,NaN


In [49]:
# mapping_candidates.to_clipboard(index=False)

## Save these easy term mappings to SQLite

In [50]:
# mapping_candidates.to_sql('mapping_scratch', biosample_cnx, if_exists='replace', index=False)

## For which Biosample annotations were no easy mappings found?
How many Biosamples uses those annotations?

In [51]:
flag = ~mapping_candidates.id_or_lab_ok
needs_search = mapping_candidates.loc[flag]

needs_search = needs_search[["remaining_tidied", "count"]]

sum_by_needed = needs_search.groupby("remaining_tidied")["count"].sum()

sum_by_needed = sum_by_needed.to_frame()
sum_by_needed["remaining_tidied"] = sum_by_needed.index

sum_by_needed = sum_by_needed.sort_values("count", ascending=False)
sum_by_needed.reset_index(drop=True)

sum_by_needed.to_sql("sum_by_needed", biosample_cnx, if_exists="replace", index=False)

In [52]:
sum_by_needed

,count,remaining_tidied
remaining_tidied,,
organic soil,162,organic soil
,113,
dna,99,dna
mine tailings,90,mine tailings
mineral soil,80,mineral soil
...,...,...
mud soil,1,mud soil
paddy soil,1,paddy soil
replicate 1 soil from baodongyu of taoyuan,1,replicate 1 soil from baodongyu of taoyuan


## Extract the tidied strings

In [53]:
ebs_raw_list = list(sum_by_needed["remaining_tidied"])
ebs_raw_list.sort()

## Submit those tidied strings to a search engine

Specifically OLS search. This takes roughly one second per unique post-tidied submission

_Turn logging back on to show status?_
_Print the count and pre- and post- datestamps_


In [54]:
ebs_search_res = scoped_mapping.search_get_annotations_wrapper(
    ebs_raw_list,
    bad_chars=chars_to_whiteout,
    cat_name=biosample_col_to_map,
    ontoprefix=first_pass_ontologies,
    query_fields="",
    rr=5,
    string_dist_arg=my_string_dist_arg,
)
ebs_search_res.to_sql(
    "ols_search_results", biosample_cnx, if_exists="replace", index=False
)

## Filter out the best of the acceptable mappings
From a string distance perspective

In [55]:
my_best_acceptable = scoped_mapping.get_best_acceptable(
    ebs_search_res, max_string_dist=my_max_string_dist
)
my_best_acceptable

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
45,env_medium,bacteria,bacteria,Bacteria,1,0.000,NCBITaxon:2,Bacteria,1,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_2,ncbitaxon
181,env_medium,mine tailings,mine tailings,mine tailing,1,0.036,ENVO:00000003,mine tailing,4,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00000003,envo
301,env_medium,peat,peat,peat,1,0.000,ENVO:00005774,peat soil,1,ENVO,hasRelatedSynonym,None,http://purl.obolibrary.org/obo/ENVO_00005774,envo
424,env_medium,sandy soil,sandy soil,sandy soil,1,0.000,ENVO:00002229,arenosol,1,ENVO,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_00002229,envo


----

## Filter out the submissions with no acceptable matches

In [56]:
no_acceptable_mappings = scoped_mapping.get_no_acceptable_mappings(
    ebs_search_res, my_best_acceptable
)

no_acceptable_mappings

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
0,env_medium,,,,1,0.000,,,1,,NaN,NaN,,
1,env_medium,13,13,bacterium A9 13-13,1,0.541,NCBITaxon:1469932,bacterium A9 13-13,1,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1469932,ncbitaxon
3,env_medium,13,13,Phage 13,2,0.622,NCBITaxon:12424,Phage 13,3,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_12424,ncbitaxon
2,env_medium,13,13,Pasteurellaceae bacterium 13/13,3,0.667,NCBITaxon:1347399,Pasteurellaceae bacterium 13/13,2,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1347399,ncbitaxon
4,env_medium,13,13,Bacteriophage 13,4,0.742,NCBITaxon:12424,Phage 13,3,NCBITAXON,hasExactSynonym,equivalent name,http://purl.obolibrary.org/obo/NCBITaxon_12424,ncbitaxon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,env_medium,topsoil,topsoil,soil erosion,1,0.631,ENVO:01000706,soil erosion,3,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_01000706,envo
519,env_medium,topsoil,topsoil,luvisol,2,0.833,ENVO:00002248,luvisol,4,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00002248,envo
520,env_medium,topsoil,topsoil,lixisol,3,0.833,ENVO:00002242,lixisol,5,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00002242,envo
517,env_medium,topsoil,topsoil,stagnosol,4,0.856,ENVO:00002274,stagnosol,2,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00002274,envo


## Try searching the failures against all ontologies in OLS
what would it be like to use a local OLS instance? (latency-wise)

In [57]:
still_unmapped = list(set(list(no_acceptable_mappings["raw"])))
still_unmapped.sort()

print(len(still_unmapped))

still_unmapped

61


['',
 '13',
 '15',
 '16s rrna gene sequence',
 '22',
 '24',
 '4',
 '6',
 'alfisols',
 'biomat',
 'cryoconite',
 'decomposed litter oe layer',
 'dna',
 'fermentation humic sustrate',
 'forest soil biochar',
 'genome',
 'genomic bacterial dna of the ssu rrna gene v5 v7 hypervariable region',
 'hypolith scappings from under rocks',
 'juvenile from soil aluvial loam',
 'juvenile from soil diluvial sand',
 'juvenile from soil loess loam',
 'juveniles meloidogyne hapla from soil diluvial sand',
 'leaves',
 'litters',
 'mineral soil',
 'mixed fungi',
 'molecular ecology sample w1 3',
 'molecular ecology sample w1 6',
 'molecular ecology sample x1 3',
 'molecular ecology sample x1 6',
 'moss',
 'mud soil',
 'not applicable',
 'organic soil',
 'paddy soil',
 'permafrost soil',
 'picea litter',
 'pinus litter',
 'replicate 1 soil from baodongyu of taoyuan',
 'replicate 1 soil from gushi of taoyuan',
 'replicate 1 soil from jiaxing',
 'replicate 1 soil from leizhou',
 'replicate 1 soil from yingt

In [58]:

salvage_search_res = scoped_mapping.search_get_annotations_wrapper(
    still_unmapped,
    bad_chars="._-",
    cat_name="salvage",
    ontoprefix="",
    query_fields="",
    rr=5,
    string_dist_arg=2,
)

salvage_search_res.to_sql(
    "salvage_search_res", biosample_cnx, if_exists="replace", index=False
)

## We appear to be at a point of diminishing returns
At the very least, will require some review

In [59]:
salvage_best_acceptable = scoped_mapping.get_best_acceptable(
    salvage_search_res, max_string_dist=salvage_max_dist
)
salvage_best_acceptable

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
11,salvage,15,15,15,1,0.000,NCIT:C113429,Fifteen,5,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C113429,ncit
73,salvage,4,4,4,1,0.000,NCIT:C172977,Fear Score 4,5,NCIT,hasExactSynonym,None,http://purl.obolibrary.org/obo/NCIT_C172977,ncit
88,salvage,6,6,6,1,0.000,NCIT:C179016,COVID-19 Disease Severity 6,3,NCIT,hasExactSynonym,None,http://purl.obolibrary.org/obo/NCIT_C179016,ncit
107,salvage,cryoconite,cryoconite,cryoconite hole,1,0.198,ENVO:03000039,cryoconite hole,2,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_03000039,envo
125,salvage,dna,dna,DNA,1,0.000,NCIT:C449,DNA,1,NCIT,label,label,http://purl.obolibrary.org/obo/NCIT_C449,ncit
162,salvage,genome,genome,Genome,1,0.000,NCIT:C16629,Genome,1,NCIT,label,label,http://purl.obolibrary.org/obo/NCIT_C16629,ncit
291,salvage,leaves,leaves,leaves,1,0.000,CO_366:0000273,leaves,1,CO_366,label,label,http://www.cropontology.org/rdf/CO_366:0000273,co_366
520,salvage,not applicable,not applicable,Not Applicable,1,0.000,NCIT:C48660,Not Applicable,1,NCIT,label,label,http://purl.obolibrary.org/obo/NCIT_C48660,ncit
576,salvage,paddy soil,paddy soil,rice paddy soil,1,0.198,ENVO:00005740,paddy field soil,1,ENVO,has_related_synonym,annotation,http://purl.obolibrary.org/obo/ENVO_00005740,envo
588,salvage,permafrost soil,permafrost soil,permafrost,1,0.198,ENVO:00000134,permafrost,1,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00000134,envo


## Save the mappings for review!

_Note that the merge on ID or label mappings are not included here_


In [60]:
my_best_acceptable[include_col] = first_pass_include_val
salvage_best_acceptable[include_col] = salvage_include_val
best_and_salvage = pd.concat([my_best_acceptable, salvage_best_acceptable])

best_and_salvage["ols_curation_notes"] = ""

best_and_salvage.to_csv(ols_review_file, sep=ols_review_seperator, index=False)

In [61]:
best_and_salvage.to_clipboard(index=False)

In [62]:
raise SystemExit("Don't skip reviewing and saving!")

SystemExit: Don't skip reviewing and saving!

/Users/MAM/Documents/gitrepos/scoped-mapping/sm_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Do some review in a spreadsheet application!
Specifically, columns `include` and `ols_curation_notes`

Then Save

----

## Now read the reviewed spreadsheet back in

In [ ]:
curated = pd.read_csv(ols_review_file, sep=ols_review_seperator)
curated

## OR copy and paste

In [63]:

curated = pd.read_clipboard()
curated

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name,include
0,salvage,leaves,leaves,leaves,1,0.000,CO_366:0000273,leaves,1,CO_366,label,label,http://www.cropontology.org/rdf/CO_366:0000273,co_366,False
1,salvage,cryoconite,cryoconite,cryoconite hole,1,0.198,ENVO:03000039,cryoconite hole,2,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_03000039,envo,False
2,salvage,paddy soil,paddy soil,rice paddy soil,1,0.198,ENVO:00005740,paddy field soil,1,ENVO,has_related_synonym,annotation,http://purl.obolibrary.org/obo/ENVO_00005740,envo,True
3,salvage,permafrost soil,permafrost soil,permafrost,1,0.198,ENVO:00000134,permafrost,1,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00000134,envo,False
4,salvage,rhizosphere soil,rhizosphere soil,rhizosphere,1,0.184,ENVO:00005801,rhizosphere,1,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00005801,envo,False
5,salvage,surface sediment water,surface sediment water,sediment surface,1,0.155,ENVO:01001192,sediment surface,2,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_01001192,envo,False
6,salvage,15,15,15,1,0.000,NCIT:C113429,Fifteen,5,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C113429,ncit,False
7,salvage,4,4,4,1,0.000,NCIT:C172977,Fear Score 4,5,NCIT,hasExactSynonym,NaN,http://purl.obolibrary.org/obo/NCIT_C172977,ncit,False
8,salvage,6,6,6,1,0.000,NCIT:C179016,COVID-19 Disease Severity 6,3,NCIT,hasExactSynonym,NaN,http://purl.obolibrary.org/obo/NCIT_C179016,ncit,False
9,salvage,dna,dna,DNA,1,0.000,NCIT:C449,DNA,1,NCIT,label,label,http://purl.obolibrary.org/obo/NCIT_C449,ncit,False


## Filter on `include`

In [64]:
flag = curated["include"]
curated = curated.loc[flag]

curated.columns = "ols_" + curated.columns
curated.to_sql("curated", biosample_cnx, if_exists="replace", index=False)
curated

,ols_category,ols_raw,ols_query,ols_name,ols_string_dist_rank,ols_string_dist,ols_obo_id,ols_label,ols_search_rank,ols_ontology_prefix,ols_scope,ols_type,ols_iri,ols_ontology_name,ols_include
2,salvage,paddy soil,paddy soil,rice paddy soil,1,0.198,ENVO:00005740,paddy field soil,1,ENVO,has_related_synonym,annotation,http://purl.obolibrary.org/obo/ENVO_00005740,envo,True
13,env_medium,mine tailings,mine tailings,mine tailing,1,0.036,ENVO:00000003,mine tailing,4,ENVO,label,label,http://purl.obolibrary.org/obo/ENVO_00000003,envo,True
14,env_medium,peat,peat,peat,1,0.000,ENVO:00005774,peat soil,1,ENVO,hasRelatedSynonym,NaN,http://purl.obolibrary.org/obo/ENVO_00005774,envo,True
15,env_medium,sandy soil,sandy soil,sandy soil,1,0.000,ENVO:00002229,arenosol,1,ENVO,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_00002229,envo,True
16,env_medium,bacteria,bacteria,Bacteria,1,0.000,NCBITaxon:2,Bacteria,1,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_2,ncbitaxon,True


In [65]:
merge_search_merged = mapping_candidates.merge(
    curated, how="outer", left_on="remaining_tidied", right_on="ols_raw"
)

In [66]:
flag = (
    ~merge_search_merged["id_or_lab_ok"]
    & ~merge_search_merged["ols_obo_id"].eq("")
    & ~merge_search_merged["ols_obo_id"].isna()
)

replacement = merge_search_merged.loc[flag, "ols_obo_id"]

merge_search_merged.loc[flag, "consensus_id"] = replacement

replacement = merge_search_merged.loc[flag, "ols_label"]

merge_search_merged.loc[flag, "consensus_lab"] = replacement

replacement = (
    merge_search_merged.loc[flag, "consensus_lab"]
    + " ["
    + merge_search_merged.loc[flag, "consensus_id"]
    + "]"
)

merge_search_merged.loc[flag, "assembled_consensus"] = replacement

In [67]:
merge_search_merged.to_sql(
    "merge_search_merged", biosample_cnx, if_exists="replace", index=False
)

In [68]:
summary = merge_search_merged[[biosample_col_to_map, "count", "assembled_consensus"]]
summary = summary.sort_values("count", ascending=False)
summary

,env_medium,count,assembled_consensus
0,soil,12400,soil [ENVO:00001998]
1,Soil,439,soil [ENVO:00001998]
3,rhizosphere,238,rhizosphere [ENVO:00005801]
5,peat soil,224,peat soil [ENVO:00005774]
2,ENVO:soil,214,soil [ENVO:00001998]
...,...,...,...
82,replicate 1 soil from Yingtan,1,NaN
83,replicate 1 soil from Leizhou,1,NaN
84,replicate 1 soil from Jiaxing,1,NaN
85,replicate 1 soil from Gushi of Taoyuan,1,NaN


In [69]:
frame_for_sssom = merge_search_merged

na_flag = frame_for_sssom["splitted"].isna()
frame_for_sssom = frame_for_sssom[~na_flag]

na_flag = frame_for_sssom["consensus_id"].isna()
frame_for_sssom = frame_for_sssom[~na_flag]

raw_queries = list(frame_for_sssom["splitted"])
urlencodeds = list(map(urllib.parse.quote, raw_queries))
urlencodeds = [sssom_subject_prefix + ":" + item for item in urlencodeds]

results_rows = len(frame_for_sssom.index)
iso8601_stamp = datetime.now().replace(microsecond=0).isoformat()

sssom_frame = {
    "subject_category": [biosample_col_to_map] * results_rows,
    "subject_label": frame_for_sssom["splitted"],
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "object_id": frame_for_sssom["consensus_id"],
    "object_label": frame_for_sssom["consensus_lab"],
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    "mapping_date": [iso8601_stamp] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
    "subject_id": urlencodeds,
}

sssom_frame = pd.DataFrame(sssom_frame)

sssom_frame.to_csv(sssom_file, sep=ols_review_seperator, index=False)

sssom_frame

,subject_category,subject_label,predicate_id,object_id,object_label,match_type,creator_id,mapping_date,subject_id
0,env_medium,soil,skos:relatedMatch,ENVO:00001998,soil,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:soil
1,env_medium,Soil,skos:relatedMatch,ENVO:00001998,soil,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:Soil
2,env_medium,ENVO:soil,skos:relatedMatch,ENVO:00001998,soil,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:ENVO%3Asoil
3,env_medium,rhizosphere,skos:relatedMatch,ENVO:00005801,rhizosphere,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:rhizosphere
4,env_medium,Rhizosphere,skos:relatedMatch,ENVO:00005801,rhizosphere,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:Rhizosphere
5,env_medium,peat soil,skos:relatedMatch,ENVO:00005774,peat soil,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:peat%20soil
6,env_medium,Peat soil,skos:relatedMatch,ENVO:00005774,peat soil,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:Peat%20soil
7,env_medium,sand,skos:relatedMatch,ENVO:01000017,sand,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:sand
9,env_medium,Leafy wood soil,skos:relatedMatch,ENVO:00005783,leafy wood soil,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:Leafy%20wood%20soil
10,env_medium,surface soil,skos:relatedMatch,ENVO:02000059,surface soil,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T11:05:31,biosample_ebs:surface%20soil


# From here down requires refactoring and generalization 

That may even be true for some of the blocks above this!

## Join together pipe-delimited env_broad_scale mappings 
_Any other constraints? If part_count is > 1, do we need to check for empty or null env_broad_scale, repeated or splitted?_

In [70]:
multi_part_flag = merge_search_merged["part_count"] > 1
multi_part_flag.value_counts()

False    106
Name: part_count, dtype: int64

In [71]:
multi_part_frame = merge_search_merged.loc[multi_part_flag]

multi_part_frame = multi_part_frame[
    [
        biosample_col_to_map,
        "count",
        "repeated",
        "splitted",
        "part_count",
        "seq",
        "consensus_id",
        "consensus_lab",
    ]
]

multi_part_frame = multi_part_frame.sort_values(
    [biosample_col_to_map, "seq"], ascending=(True, True)
)

multi_part_frame

,env_medium,count,repeated,splitted,part_count,seq,consensus_id,consensus_lab


In [72]:
part_count = multi_part_frame.groupby([biosample_col_to_map], sort=False)[
    "part_count"
].max()
part_count.value_counts()

Series([], Name: part_count, dtype: int64)

In [73]:
seq_max = multi_part_frame.groupby([biosample_col_to_map], sort=False)["seq"].max()
seq_max

Series([], Name: seq, dtype: float64)

In [74]:
parts_check = pd.DataFrame(dict(part_count=part_count, seq_max=seq_max)).reset_index()

all_parts_flag = parts_check["seq_max"] == parts_check["part_count"]
ebs_with_all_parts = parts_check.loc[all_parts_flag, biosample_col_to_map]
ebs_with_all_parts

all_parts_flag = multi_part_frame[biosample_col_to_map].isin(list(ebs_with_all_parts))
all_parts_frame = multi_part_frame.loc[all_parts_flag]
all_parts_frame

,env_medium,count,repeated,splitted,part_count,seq,consensus_id,consensus_lab


In [75]:
unique_pipesep_ebs = list(set(list(all_parts_frame[biosample_col_to_map])))
unique_pipesep_ebs.sort()
repipe_dict_list = []
for one_pipesep in unique_pipesep_ebs:
    flag = all_parts_frame[biosample_col_to_map] == one_pipesep
    temp_frame = all_parts_frame.loc[flag]
    ids_list = list(temp_frame["consensus_id"])
    lab_list = list(temp_frame["consensus_lab"])
    s = "|"
    ids_repipe = s.join(ids_list)
    lab_repipe = s.join(lab_list)

    repipe_dict = {
        biosample_col_to_map: one_pipesep,
        "consensus_id": ids_repipe,
        "consensus_lab": lab_repipe,
    }
    repipe_dict_list.append(repipe_dict)
repipe_frame = pd.DataFrame(repipe_dict_list)
repipe_frame

""


In [76]:
sssom_min_cols = sssom_frame[["subject_label", "object_id", "object_label"]]

sssom_min_cols.columns = [biosample_col_to_map, "consensus_id", "consensus_lab"]

sssom_min_cols

,env_medium,consensus_id,consensus_lab
0,soil,ENVO:00001998,soil
1,Soil,ENVO:00001998,soil
2,ENVO:soil,ENVO:00001998,soil
3,rhizosphere,ENVO:00005801,rhizosphere
4,Rhizosphere,ENVO:00005801,rhizosphere
5,peat soil,ENVO:00005774,peat soil
6,Peat soil,ENVO:00005774,peat soil
7,sand,ENVO:01000017,sand
9,Leafy wood soil,ENVO:00005783,leafy wood soil
10,surface soil,ENVO:02000059,surface soil


In [77]:
repipe_rows = len(repipe_frame.index)
if repipe_rows > 0:
    direct_and_repipe = pd.concat([temp, repipe_frame])
else:
    direct_and_repipe = sssom_min_cols
direct_and_repipe

,env_medium,consensus_id,consensus_lab
0,soil,ENVO:00001998,soil
1,Soil,ENVO:00001998,soil
2,ENVO:soil,ENVO:00001998,soil
3,rhizosphere,ENVO:00005801,rhizosphere
4,Rhizosphere,ENVO:00005801,rhizosphere
5,peat soil,ENVO:00005774,peat soil
6,Peat soil,ENVO:00005774,peat soil
7,sand,ENVO:01000017,sand
9,Leafy wood soil,ENVO:00005783,leafy wood soil
10,surface soil,ENVO:02000059,surface soil


In [78]:
direct_and_repipe.to_sql(
    "direct_and_repipe", biosample_cnx, if_exists="replace", index=False
)

## refactor this even more?


In [79]:
q = (
    """
select
	b.id, b."""
    + biosample_col_to_map
    + """,
	dar.consensus_id ,
	dar.consensus_lab
from
	direct_and_repipe dar
left join biosample b on
	b."""
    + biosample_col_to_map
    + """ = dar."""
    + biosample_col_to_map
    + """
inner join biobiosample_tax_id_counts stic on
	b.taxonomy_id = stic.biosample_taxid
join env_package_normalized on
	b.env_package = env_package_normalized.env_package
where """
    + scoping_col
    + " = '"
    + scoping_value
    + "'"
)

[per_biosample_scoped_mapping_results, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

2021-06-11 11:05:31.459113
2021-06-11 11:05:31.667968
0:00:00.208855


In [80]:
per_biosample_scoped_mapping_results["notes"] = scoping_col + "=" + scoping_value

table_name = "repaired_" + biosample_col_to_map

per_biosample_scoped_mapping_results.to_sql(
    table_name, biosample_cnx, if_exists="append", index=False
)

per_biosample_scoped_mapping_results

,id,env_medium,consensus_id,consensus_lab,notes
0,BIOSAMPLE:SAMEA1706552,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
1,BIOSAMPLE:SAMEA1706550,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
2,BIOSAMPLE:SAMEA1706543,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
3,BIOSAMPLE:SAMEA1706553,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
4,BIOSAMPLE:SAMEA1706542,greenhouse soil,ENVO:00005780,greenhouse soil,env_package_normalized.EnvPackage=soil
...,...,...,...,...,...
14946,BIOSAMPLE:SAMEA1687814,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
14947,BIOSAMPLE:SAMEA1687805,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
14948,BIOSAMPLE:SAMEA1687808,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
14949,BIOSAMPLE:SAMEA1687809,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil


In [81]:
per_biosample_scoped_mapping_results.to_csv(
    per_biosample_mapping_results_file,
    sep=ols_review_seperator,
    index=False,
)

In [ ]:
raise SystemExit("Manually do ebs, els and em, OR AUTOMTE!")

----

In [108]:
# hard to do with pure sql in sqlite3 (no full outer jons,
# just nested unions https://www.sqlitetutorial.net/sqlite-full-outer-join/)

# refactor


q = "select * from repaired_env_package where env_package_rep = 'soil'"

[repaired_env_package, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

q = "select * from repaired_env_broad_scale"

[repaired_env_broad_scale, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

repaired_env_broad_scale.drop("notes", axis=1)
colgroup = '_ebs'
repaired_env_broad_scale=repaired_env_broad_scale.rename(columns = {'consensus_id':'consensus_id'+colgroup})
repaired_env_broad_scale=repaired_env_broad_scale.rename(columns = {'consensus_lab':'consensus_lab'+colgroup})
# repaired_env_broad_scale.columns[2] = repaired_env_broad_scale.columns[2] + colgroup
# repaired_env_broad_scale.columns[3] = repaired_env_broad_scale.columns[3] + colgroup

# temp = repaired_env_broad_scale.columns
# temp = [sub + colgroup for sub in temp]
# temp[0] = 'id'
# repaired_env_broad_scale.columns = temp

q = "select * from repaired_env_local_scale"

[repaired_env_local_scale, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

repaired_env_local_scale.drop("notes", axis=1)
colgroup = '_els'
repaired_env_local_scale=repaired_env_local_scale.rename(columns = {'consensus_id':'consensus_id'+colgroup})
repaired_env_local_scale=repaired_env_local_scale.rename(columns = {'consensus_lab':'consensus_lab'+colgroup})


q = "select * from repaired_env_medium"

[repaired_env_medium, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

colgroup = '_em'
repaired_env_medium=repaired_env_medium.rename(columns = {'consensus_id':'consensus_id'+colgroup})
repaired_env_medium=repaired_env_medium.rename(columns = {'consensus_lab':'consensus_lab'+colgroup})


2021-06-11 12:02:49.805963
2021-06-11 12:02:49.849540
0:00:00.043577
2021-06-11 12:02:49.850565
2021-06-11 12:02:49.888576
0:00:00.038011
2021-06-11 12:02:49.894644
2021-06-11 12:02:49.915513
0:00:00.020869
2021-06-11 12:02:49.918733
2021-06-11 12:02:49.956433
0:00:00.037700


In [109]:
repaired_env_medium

,id,env_medium,consensus_id_em,consensus_lab_em,notes
0,BIOSAMPLE:SAMEA1706552,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
1,BIOSAMPLE:SAMEA1706550,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
2,BIOSAMPLE:SAMEA1706543,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
3,BIOSAMPLE:SAMEA1706553,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
4,BIOSAMPLE:SAMEA1706542,greenhouse soil,ENVO:00005780,greenhouse soil,env_package_normalized.EnvPackage=soil
...,...,...,...,...,...
14946,BIOSAMPLE:SAMEA1687814,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
14947,BIOSAMPLE:SAMEA1687805,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
14948,BIOSAMPLE:SAMEA1687808,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
14949,BIOSAMPLE:SAMEA1687809,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil


In [110]:
from functools import reduce

dflist = [
    repaired_env_package,
    repaired_env_broad_scale,
    repaired_env_local_scale,
    repaired_env_medium,
]
nan_value = ""
# dfconcat = pd.concat(dflist, join='outer', axis=1).fillna(nan_value)
# dfconcat

df_merged = reduce(
    lambda left, right: pd.merge(left, right, on=["id"], how="outer"), dflist
).fillna(nan_value)

In [111]:
df_merged

,id,env_package_orig,env_package_rep,env_broad_scale,consensus_id_ebs,consensus_lab_ebs,notes_x,env_local_scale,consensus_id_els,consensus_lab_els,notes_y,env_medium,consensus_id_em,consensus_lab_em,notes
0,BIOSAMPLE:SAMEA1706488,soil,soil,,,,,,,,,,,,
1,BIOSAMPLE:SAMEA1706552,soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,anthropogenic habitat,ENVO:00002031,anthropogenic habitat,env_package_normalized.EnvPackage=soil,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
2,BIOSAMPLE:SAMEA1706550,soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,anthropogenic habitat,ENVO:00002031,anthropogenic habitat,env_package_normalized.EnvPackage=soil,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
3,BIOSAMPLE:SAMEA1706543,soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,anthropogenic habitat,ENVO:00002031,anthropogenic habitat,env_package_normalized.EnvPackage=soil,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
4,BIOSAMPLE:SAMEA1706553,soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,anthropogenic habitat,ENVO:00002031,anthropogenic habitat,env_package_normalized.EnvPackage=soil,oil contaminated soil,ENVO:00002875,oil contaminated soil,env_package_normalized.EnvPackage=soil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15808,BIOSAMPLE:SAMEA1687814,Soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,soil,ENVO:00001998,soil,env_package_normalized.EnvPackage=soil,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
15809,BIOSAMPLE:SAMEA1687805,Soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,soil,ENVO:00001998,soil,env_package_normalized.EnvPackage=soil,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
15810,BIOSAMPLE:SAMEA1687808,Soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,soil,ENVO:00001998,soil,env_package_normalized.EnvPackage=soil,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil
15811,BIOSAMPLE:SAMEA1687809,Soil,soil,terrestrial biome,ENVO:00000446,terrestrial biome,env_package_normalized.EnvPackage=soil,soil,ENVO:00001998,soil,env_package_normalized.EnvPackage=soil,arable soil,ENVO:00005742,arable soil,env_package_normalized.EnvPackage=soil


In [112]:
df_merged.to_csv('repaired_soil_packages.tsv', sep='\t', index=False)